<a href="https://colab.research.google.com/github/Shaurya-d-12345/AnxietyClassif/blob/main/eegproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [114]:
import scipy.io
import pandas as pd
import numpy as np
from scipy.io import loadmat
import os

In [115]:
path = "/content/drive/MyDrive/Data/filtered_data"
data_list = []
for filename in os.listdir(path):
    if filename.endswith('.mat') and not filename.startswith('Relax'):
        filepath = os.path.join(path, filename)
        data = scipy.io.loadmat(filepath)
        df = pd.DataFrame(data['Clean_data'])
        data_list.append(df)

df = pd.concat(data_list)

# df.to_csv('/content/csv/file.csv', index=False)

In [116]:
cdf = pd.DataFrame()
df = df.reset_index(drop=True)

# print(df)


In [117]:
%%capture
df.columns = df.columns.map(str)
# for i in range(len(df)):
#     if i%32== 1 or i%32==2 or i%32==11 or i%32== 12 or i%32== 14 or i%32== 15:
#       cdf.columns = cdf.columns.map(str)
#       cdf = cdf.append(df.loc[i])
cdf = df
cdf = cdf.reset_index(drop=True)
print(cdf)



In [118]:
print(cdf)

              0          1         2          3          4         5  \
0      0.500690   1.627487 -4.121549  -8.307720  -1.825085 -1.532250   
1      0.345449  -1.948893 -2.746191  -2.126710  -2.147028 -3.720288   
2      4.421423  -1.442802  0.508761   2.202509  -0.694570 -5.002202   
3      4.101206  -1.619572 -1.165751   1.664365   1.667636 -3.412924   
4      3.695614  -2.665014 -5.724447   0.832410   2.050910 -3.786971   
...         ...        ...       ...        ...        ...       ...   
11515  0.146356  -5.582920 -7.193353  -6.806170  -7.430401 -4.192025   
11516 -1.042550  -5.945982 -8.196780  -7.383520  -6.378967 -0.715280   
11517 -1.105353  -7.615806 -5.236382  -1.551307  -6.380616 -3.791196   
11518 -0.887582 -10.421530 -3.521661   2.788679  -9.491486 -8.400833   
11519 -4.945934  -7.944150 -9.940305 -15.011743 -21.122254 -2.239738   

               6          7          8          9  ...       3190      3191  \
0      -2.982746   6.514606   6.589658  -4.650158  ...  

In [119]:
#hjorth parameters



def extract_hjorth_features(cdf):
    #  first-order Hjorth parameters
    activity = cdf.var(axis=1)
    mobility = np.sqrt(cdf.diff(axis=1).var(axis=1) / cdf.var(axis=1))
    complexity = (np.sqrt(cdf.diff(axis=1).diff(axis=1).var(axis=1) / cdf.diff(axis=1).var(axis=1)) 
                  / np.sqrt(cdf.diff(axis=1).var(axis=1) / cdf.var(axis=1)))
    
    
    return {'Activity': activity,
            'Mobility': mobility,
            'Complexity': complexity
            }


dict = extract_hjorth_features(cdf)
# print(dict)
hjorth_features = pd.DataFrame.from_dict(dict)

print(hjorth_features)

         Activity  Mobility  Complexity
0       31.831900  0.919900    1.476284
1       31.418703  0.886648    1.529104
2       38.242152  0.898043    1.518281
3       37.063702  0.907317    1.501894
4       27.303752  0.939997    1.451066
...           ...       ...         ...
11515   48.524351  0.869430    1.486233
11516   54.586558  0.847464    1.513751
11517   53.141926  0.865656    1.494438
11518   73.049697  0.934720    1.431812
11519  117.630419  0.783018    1.677801

[11520 rows x 3 columns]


In [120]:
pip install nolds

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [121]:
# FFT-based band power 
from scipy.signal import welch
fs = 256  # sampling rate

# Reshape the EEG data to a 3D array with dimensions (num_subjects, num_channels, num_samples)
eeg_data = cdf.values.reshape((11520, 1, 3200))
# print(eeg_data)

# Compute the FFT of the EEG data
fs = 256  # sampling rate
f, psd = welch(eeg_data, fs, nperseg=fs*2, axis=-1)

# Compute the power spectrum of the EEG data
psd = np.square(np.abs(psd))

# Define frequency bands of interest
bands = {'Delta': (0, 4),
         'Theta': (4, 8),
         'Alpha': (8, 13),
         'Beta': (13, 30),
         'Gamma': (30, 100)}

# Compute the power in each frequency band for each subject
band_power = {}
for band_name, band_limits in bands.items():
    freq_ix = np.where((f >= band_limits[0]) & (f < band_limits[1]))[0]
    band_power[band_name] = np.sum(psd[..., freq_ix], axis=-1)
# print(band_power)
for i in band_power:
     band_power[i] = band_power[i].flatten()
     band_power[i] = pd.Series(band_power[i])

# band_power_df = pd.DataFrame.from_dict(band_power)
band_power_df = pd.DataFrame.from_dict(band_power)
print(band_power_df)

          Delta       Theta      Alpha        Beta      Gamma
0      0.149434   11.663945   3.865947   12.637414   7.785473
1      0.195005   26.761211   5.143719    8.055040   7.143923
2      0.463014   58.721811   8.399288    8.957267  11.870116
3      0.513686   50.386970   7.364822    8.359088  11.378229
4      0.129092   15.423091   2.848433    6.183781   6.878625
...         ...         ...        ...         ...        ...
11515  0.288369   40.450824   8.815813   23.986074  19.323733
11516  0.428790   55.854137  12.669003   31.871685  22.948608
11517  0.336398   58.245148  11.076394   26.317758  22.568554
11518  0.805546  107.115217  19.802565   33.210211  50.555472
11519  2.739948  746.703433  60.038561  108.219753  68.673872

[11520 rows x 5 columns]


In [122]:
# DWT based RMS features
import pywt

# Reshape the EEG data to a 3D array with dimensions (num_subjects, num_channels, num_samples)
eeg_data = cdf.values.reshape((11520, 1, 3200))

# Define the wavelet family and decomposition level
wavelet = 'db4'
level = 5

# Compute the DWT of the EEG data
coeffs = pywt.wavedec(eeg_data, wavelet, level=level, axis=-1)

# Compute the RMS of the DWT coefficients
rms_coeffs = [pywt.threshold(c, value=0.1, mode='soft') for c in coeffs]
rms = pywt.waverec(rms_coeffs, wavelet, axis=-1)

# Compute the RMS features for each subject
rms_features = rms.mean(axis=-1)

# Store the RMS features in a new DataFrame
rms_df = pd.DataFrame(rms_features, columns=['rms'])

print(rms_df)

            rms
0      0.066858
1      0.045517
2      0.028886
3     -0.006770
4     -0.037441
...         ...
11515 -0.067352
11516 -0.075988
11517 -0.036833
11518 -0.011128
11519  0.070168

[11520 rows x 1 columns]


In [123]:
hjorth_features.index.name = 'Subject'
band_power_df.index.name = 'Subject'
rms_df.index.name = 'Subject'
uldf = [hjorth_features, band_power_df, rms_df]
# uldf = [df.set_index('') for df in uldf]
uldf[0].join(uldf[1:])

,Activity,Mobility,Complexity,Delta,Theta,Alpha,Beta,Gamma,rms
Subject,,,,,,,,,
0,31.831900,0.919900,1.476284,0.149434,11.663945,3.865947,12.637414,7.785473,0.066858
1,31.418703,0.886648,1.529104,0.195005,26.761211,5.143719,8.055040,7.143923,0.045517
2,38.242152,0.898043,1.518281,0.463014,58.721811,8.399288,8.957267,11.870116,0.028886
3,37.063702,0.907317,1.501894,0.513686,50.386970,7.364822,8.359088,11.378229,-0.006770
4,27.303752,0.939997,1.451066,0.129092,15.423091,2.848433,6.183781,6.878625,-0.037441
...,...,...,...,...,...,...,...,...,...
11515,48.524351,0.869430,1.486233,0.288369,40.450824,8.815813,23.986074,19.323733,-0.067352
11516,54.586558,0.847464,1.513751,0.428790,55.854137,12.669003,31.871685,22.948608,-0.075988
11517,53.141926,0.865656,1.494438,0.336398,58.245148,11.076394,26.317758,22.568554,-0.036833


In [124]:
uldf1 = pd.DataFrame(uldf[0].join(uldf[1:]))
print(uldf1)

           Activity  Mobility  Complexity     Delta       Theta      Alpha  \
Subject                                                                      
0         31.831900  0.919900    1.476284  0.149434   11.663945   3.865947   
1         31.418703  0.886648    1.529104  0.195005   26.761211   5.143719   
2         38.242152  0.898043    1.518281  0.463014   58.721811   8.399288   
3         37.063702  0.907317    1.501894  0.513686   50.386970   7.364822   
4         27.303752  0.939997    1.451066  0.129092   15.423091   2.848433   
...             ...       ...         ...       ...         ...        ...   
11515     48.524351  0.869430    1.486233  0.288369   40.450824   8.815813   
11516     54.586558  0.847464    1.513751  0.428790   55.854137  12.669003   
11517     53.141926  0.865656    1.494438  0.336398   58.245148  11.076394   
11518     73.049697  0.934720    1.431812  0.805546  107.115217  19.802565   
11519    117.630419  0.783018    1.677801  2.739948  746.703433 

In [125]:
%%capture
df0 = pd.read_excel('/content/drive/MyDrive/labels.xlsx')
# print(df0)
df0 = df0.set_index('Subject No.')
df2 = pd.DataFrame()
df0 = df0.reset_index(drop = True)
k = 0
for i in range(len(df0)):
  for j in range(len(df0)):
    if j%40 == k:
      df2 = df2.append(df0.loc[j])
  k = k+1
# print(df2)

In [126]:
df2 = df2.reset_index(drop = True)


In [127]:
%%capture
labeled = pd.read_csv('/content/drive/MyDrive/label.csv')
labeled = labeled.set_index('Subject')
labeled = labeled.reset_index(drop = True)
final = pd.DataFrame()
for i in range(len(labeled)):
  final = final.append(labeled.loc[i])
  final = final.append(labeled.loc[i])
  final = final.append(labeled.loc[i])
  final = final.append(labeled.loc[i])
  final = final.append(labeled.loc[i])
  final = final.append(labeled.loc[i])
  final = final.append(labeled.loc[i])
  final = final.append(labeled.loc[i])
  final = final.append(labeled.loc[i])
  final = final.append(labeled.loc[i])
  final = final.append(labeled.loc[i])
  final = final.append(labeled.loc[i])
  final = final.append(labeled.loc[i])
  final = final.append(labeled.loc[i])
  final = final.append(labeled.loc[i])
  final = final.append(labeled.loc[i])
  final = final.append(labeled.loc[i])
  final = final.append(labeled.loc[i])
  final = final.append(labeled.loc[i])
  final = final.append(labeled.loc[i])
  final = final.append(labeled.loc[i])
  final = final.append(labeled.loc[i])
  final = final.append(labeled.loc[i])
  final = final.append(labeled.loc[i])
  final = final.append(labeled.loc[i])
  final = final.append(labeled.loc[i])
  final = final.append(labeled.loc[i])
  final = final.append(labeled.loc[i])
  final = final.append(labeled.loc[i])
  final = final.append(labeled.loc[i])
  final = final.append(labeled.loc[i])
  final = final.append(labeled.loc[i])



In [ ]:
# %%capture
# uldf2 = uldf1
# uldf2['Activity1'] = ''
# uldf2['Mobility1'] = ''
# uldf2['Complexity1'] = ''
# uldf2['Delta1'] = ''
# uldf2['Theta1'] = ''
# uldf2['Alpha1'] = ''
# uldf2['Beta1'] = ''
# uldf2['Gamma1'] = ''
# uldf2['rms1'] = ''
# uldf2['Activity2'] = ''
# uldf2['Mobility2'] = ''
# uldf2['Complexity2'] = ''
# uldf2['Delta2'] = ''
# uldf2['Theta2'] = ''
# uldf2['Alpha2'] = ''
# uldf2['Beta2'] = ''
# uldf2['Gamma2'] = ''
# uldf2['rms2'] = ''
# uldf2['Activity3'] = ''
# uldf2['Mobility3'] = ''
# uldf2['Complexity3'] = ''
# uldf2['Delta3'] = ''
# uldf2['Theta3'] = ''
# uldf2['Alpha3'] = ''
# uldf2['Beta3'] = ''
# uldf2['Gamma3'] = ''
# uldf2['rms3'] = ''
# uldf2['Activity4'] = ''
# uldf2['Mobility4'] = ''
# uldf2['Complexity4'] = ''
# uldf2['Delta4'] = ''
# uldf2['Theta4'] = ''
# uldf2['Alpha4'] = ''
# uldf2['Beta4'] = ''
# uldf2['Gamma4'] = ''
# uldf2['rms4'] = ''
# uldf2['Activity5'] = ''
# uldf2['Mobility5'] = ''
# uldf2['Complexity5'] = ''
# uldf2['Delta5'] = ''
# uldf2['Theta5'] = ''
# uldf2['Alpha5'] = ''
# uldf2['Beta5'] = ''
# uldf2['Gamma5'] = ''
# uldf2['rms5'] = ''
# uldf2['Activity6'] = ''
# uldf2['Mobility6'] = ''
# uldf2['Complexity6'] = ''
# uldf2['Delta6'] = ''
# uldf2['Theta6'] = ''
# uldf2['Alpha6'] = ''
# uldf2['Beta6'] = ''
# uldf2['Gamma6'] = ''
# uldf2['rms6'] = ''
# uldf2['Activity7'] = ''
# uldf2['Mobility7'] = ''
# uldf2['Complexity7'] = ''
# uldf2['Delta7'] = ''
# uldf2['Theta7'] = ''
# uldf2['Alpha7'] = ''
# uldf2['Beta7'] = ''
# uldf2['Gamma7'] = ''
# uldf2['rms7'] = ''
# uldf2['Activity8'] = ''
# uldf2['Mobility8'] = ''
# uldf2['Complexity8'] = ''
# uldf2['Delta8'] = ''
# uldf2['Theta8'] = ''
# uldf2['Alpha8'] = ''
# uldf2['Beta8'] = ''
# uldf2['Gamma8'] = ''
# uldf2['rms8'] = ''
# uldf2['Activity9'] = ''
# uldf2['Mobility9'] = ''
# uldf2['Complexity9'] = ''
# uldf2['Delta9'] = ''
# uldf2['Theta9'] = ''
# uldf2['Alpha9'] = ''
# uldf2['Beta9'] = ''
# uldf2['Gamma9'] = ''
# uldf2['rms9'] = ''
# uldf2['Activity10'] = ''
# uldf2['Mobility10'] = ''
# uldf2['Complexity10'] = ''
# uldf2['Delta10'] = ''
# uldf2['Theta10'] = ''
# uldf2['Alpha10'] = ''
# uldf2['Beta10'] = ''
# uldf2['Gamma10'] = ''
# uldf2['rms10'] = ''
# uldf2['Activity11'] = ''
# uldf2['Mobility11'] = ''
# uldf2['Complexity11'] = ''
# uldf2['Delta11'] = ''
# uldf2['Theta11'] = ''
# uldf2['Alpha11'] = ''
# uldf2['Beta11'] = ''
# uldf2['Gamma11'] = ''
# uldf2['rms11'] = ''
# uldf2['Activity12'] = ''
# uldf2['Mobility12'] = ''
# uldf2['Complexity12'] = ''
# uldf2['Delta12'] = ''
# uldf2['Theta12'] = ''
# uldf2['Alpha12'] = ''
# uldf2['Beta12'] = ''
# uldf2['Gamma12'] = ''
# uldf2['rms12'] = ''
# uldf2['Activity13'] = ''
# uldf2['Mobility13'] = ''
# uldf2['Complexity13'] = ''
# uldf2['Delta13'] = ''
# uldf2['Theta13'] = ''
# uldf2['Alpha13'] = ''
# uldf2['Beta13'] = ''
# uldf2['Gamma13'] = ''
# uldf2['rms13'] = ''
# uldf2['Activity14'] = ''
# uldf2['Mobility14'] = ''
# uldf2['Complexity14'] = ''
# uldf2['Delta14'] = ''
# uldf2['Theta14'] = ''
# uldf2['Alpha14'] = ''
# uldf2['Beta14'] = ''
# uldf2['Gamma14'] = ''
# uldf2['rms14'] = ''
# uldf2['Activity15'] = ''
# uldf2['Mobility15'] = ''
# uldf2['Complexity15'] = ''
# uldf2['Delta15'] = ''
# uldf2['Theta15'] = ''
# uldf2['Alpha15'] = ''
# uldf2['Beta15'] = ''
# uldf2['Gamma15'] = ''
# uldf2['rms15'] = ''
# uldf2['Activity16'] = ''
# uldf2['Mobility16'] = ''
# uldf2['Complexity16'] = ''
# uldf2['Delta16'] = ''
# uldf2['Theta16'] = ''
# uldf2['Alpha16'] = ''
# uldf2['Beta16'] = ''
# uldf2['Gamma16'] = ''
# uldf2['rms16'] = ''
# uldf2['Activity17'] = ''
# uldf2['Mobility17'] = ''
# uldf2['Complexity17'] = ''
# uldf2['Delta17'] = ''
# uldf2['Theta17'] = ''
# uldf2['Alpha17'] = ''
# uldf2['Beta17'] = ''
# uldf2['Gamma17'] = ''
# uldf2['rms17'] = ''
# uldf2['Activity18'] = ''
# uldf2['Mobility18'] = ''
# uldf2['Complexity18'] = ''
# uldf2['Delta18'] = ''
# uldf2['Theta18'] = ''
# uldf2['Alpha18'] = ''
# uldf2['Beta18'] = ''
# uldf2['Gamma18'] = ''
# uldf2['rms18'] = ''
# uldf2['Activity19'] = ''
# uldf2['Mobility19'] = ''
# uldf2['Complexity19'] = ''
# uldf2['Delta19'] = ''
# uldf2['Theta19'] = ''
# uldf2['Alpha19'] = ''
# uldf2['Beta19'] = ''
# uldf2['Gamma19'] = ''
# uldf2['rms19'] = ''
# uldf2['Activity20'] = ''
# uldf2['Mobility20'] = ''
# uldf2['Complexity20'] = ''
# uldf2['Delta20'] = ''
# uldf2['Theta20'] = ''
# uldf2['Alpha20'] = ''
# uldf2['Beta20'] = ''
# uldf2['Gamma20'] = ''
# uldf2['rms20'] = ''
# uldf2['Activity21'] = ''
# uldf2['Mobility21'] = ''
# uldf2['Complexity21'] = ''
# uldf2['Delta21'] = ''
# uldf2['Theta21'] = ''
# uldf2['Alpha21'] = ''
# uldf2['Beta21'] = ''
# uldf2['Gamma21'] = ''
# uldf2['rms21'] = ''
# uldf2['Activity22'] = ''
# uldf2['Mobility22'] = ''
# uldf2['Complexity22'] = ''
# uldf2['Delta22'] = ''
# uldf2['Theta22'] = ''
# uldf2['Alpha22'] = ''
# uldf2['Beta22'] = ''
# uldf2['Gamma22'] = ''
# uldf2['rms22'] = ''
# uldf2['Activity23'] = ''
# uldf2['Mobility23'] = ''
# uldf2['Complexity23'] = ''
# uldf2['Delta23'] = ''
# uldf2['Theta23'] = ''
# uldf2['Alpha23'] = ''
# uldf2['Beta23'] = ''
# uldf2['Gamma23'] = ''
# uldf2['rms23'] = ''
# uldf2['Activity24'] = ''
# uldf2['Mobility24'] = ''
# uldf2['Complexity24'] = ''
# uldf2['Delta24'] = ''
# uldf2['Theta24'] = ''
# uldf2['Alpha24'] = ''
# uldf2['Beta24'] = ''
# uldf2['Gamma24'] = ''
# uldf2['rms24'] = ''
# uldf2['Activity25'] = ''
# uldf2['Mobility25'] = ''
# uldf2['Complexity25'] = ''
# uldf2['Delta25'] = ''
# uldf2['Theta25'] = ''
# uldf2['Alpha25'] = ''
# uldf2['Beta25'] = ''
# uldf2['Gamma25'] = ''
# uldf2['rms25'] = ''
# uldf2['Activity26'] = ''
# uldf2['Mobility26'] = ''
# uldf2['Complexity26'] = ''
# uldf2['Delta26'] = ''
# uldf2['Theta26'] = ''
# uldf2['Alpha26'] = ''
# uldf2['Beta26'] = ''
# uldf2['Gamma26'] = ''
# uldf2['rms26'] = ''
# uldf2['Activity27'] = ''
# uldf2['Mobility27'] = ''
# uldf2['Complexity27'] = ''
# uldf2['Delta27'] = ''
# uldf2['Theta27'] = ''
# uldf2['Alpha27'] = ''
# uldf2['Beta27'] = ''
# uldf2['Gamma27'] = ''
# uldf2['rms27'] = ''
# uldf2['Activity28'] = ''
# uldf2['Mobility28'] = ''
# uldf2['Complexity28'] = ''
# uldf2['Delta28'] = ''
# uldf2['Theta28'] = ''
# uldf2['Alpha28'] = ''
# uldf2['Beta28'] = ''
# uldf2['Gamma28'] = ''
# uldf2['rms28'] = ''
# uldf2['Activity29'] = ''
# uldf2['Mobility29'] = ''
# uldf2['Complexity29'] = ''
# uldf2['Delta29'] = ''
# uldf2['Theta29'] = ''
# uldf2['Alpha29'] = ''
# uldf2['Beta29'] = ''
# uldf2['Gamma29'] = ''
# uldf2['rms29'] = ''
# uldf2['Activity30'] = ''
# uldf2['Mobility30'] = ''
# uldf2['Complexity30'] = ''
# uldf2['Delta30'] = ''
# uldf2['Theta30'] = ''
# uldf2['Alpha30'] = ''
# uldf2['Beta30'] = ''
# uldf2['Gamma30'] = ''
# uldf2['rms30'] = ''
# uldf2['Activity31'] = ''
# uldf2['Mobility31'] = ''
# uldf2['Complexity31'] = ''
# uldf2['Delta31'] = ''
# uldf2['Theta31'] = ''
# uldf2['Alpha31'] = ''
# uldf2['Beta31'] = ''
# uldf2['Gamma31'] = ''
# uldf2['rms31'] = ''


In [ ]:
# clist = uldf2.columns.values.tolist()



In [ ]:
# print(clist)

In [ ]:
# array = uldf2.to_numpy()
# print(np.shape(array))
# print(array)

In [ ]:
# # print(array[0][12])
# for i in range(0,11520):
#   if i%32 == 0:
#     for j in range(0,288):
#       k = i + int(j/9)
#       l = j%9
#       array[i][j] = array[k][l]
      
# # print(array)
# # print(array[32][:])
# print(array)
# df = pd.DataFrame(array,columns=[])

In [ ]:
# nfdf = pd.DataFrame(array, columns=[clist])


In [ ]:
# %%capture
# finallabeled = pd.read_csv('/content/drive/MyDrive/label.csv')
# finallabeled = finallabeled.set_index('Subject')
# finallabeled = finallabeled.reset_index(drop = True)
# finalfinal = pd.DataFrame()
# for i in range(len(finallabeled)):
#   finalfinal = finalfinal.append(finallabeled.loc[i])

In [ ]:
# %%capture
# newfinal = pd.DataFrame()
# for i in range(len(nfdf)):
#   if i%32 == 0:
#     newfinal = newfinal.append(nfdf.loc[i])



In [ ]:
# newfinal = newfinal.reset_index(drop = True)
# newfinal.index.name = 'Subject'
# newfinal

In [ ]:
# finalfinal = finalfinal.reset_index(drop = True)
# finalfinal.index.name = 'Subject'


# d_ = [newfinal,finalfinal]
# dfn = pd.DataFrame(d_[0].join(d_[1:]))
# dfn

In [128]:
final = final.reset_index(drop = True)
final.index.name = 'Subject'
final
d = [uldf1,final]
# uldf = [df.set_index('') for df in uldf]
# d[0].join(d[1:])

In [129]:
d1 = pd.DataFrame(d[0].join(d[1:]))
d1

,Activity,Mobility,Complexity,Delta,Theta,Alpha,Beta,Gamma,rms,Labels
Subject,,,,,,,,,,
0,31.831900,0.919900,1.476284,0.149434,11.663945,3.865947,12.637414,7.785473,0.066858,3
1,31.418703,0.886648,1.529104,0.195005,26.761211,5.143719,8.055040,7.143923,0.045517,3
2,38.242152,0.898043,1.518281,0.463014,58.721811,8.399288,8.957267,11.870116,0.028886,3
3,37.063702,0.907317,1.501894,0.513686,50.386970,7.364822,8.359088,11.378229,-0.006770,3
4,27.303752,0.939997,1.451066,0.129092,15.423091,2.848433,6.183781,6.878625,-0.037441,3
...,...,...,...,...,...,...,...,...,...,...
11515,48.524351,0.869430,1.486233,0.288369,40.450824,8.815813,23.986074,19.323733,-0.067352,6
11516,54.586558,0.847464,1.513751,0.428790,55.854137,12.669003,31.871685,22.948608,-0.075988,6
11517,53.141926,0.865656,1.494438,0.336398,58.245148,11.076394,26.317758,22.568554,-0.036833,6


In [130]:
d1['NLabel'] = pd.cut(x=d1['Labels'], bins=[0, 3, 5, 8, 10],
                     labels=[1, 2, 3, 4])
# print(d1)

print("Categories: ")
print(d1['NLabel'].value_counts())
d1 = d1.drop('Labels',axis= 'columns')
d1

Categories: 
2    4192
3    3616
1    3264
4     448
Name: NLabel, dtype: int64


,Activity,Mobility,Complexity,Delta,Theta,Alpha,Beta,Gamma,rms,NLabel
Subject,,,,,,,,,,
0,31.831900,0.919900,1.476284,0.149434,11.663945,3.865947,12.637414,7.785473,0.066858,1
1,31.418703,0.886648,1.529104,0.195005,26.761211,5.143719,8.055040,7.143923,0.045517,1
2,38.242152,0.898043,1.518281,0.463014,58.721811,8.399288,8.957267,11.870116,0.028886,1
3,37.063702,0.907317,1.501894,0.513686,50.386970,7.364822,8.359088,11.378229,-0.006770,1
4,27.303752,0.939997,1.451066,0.129092,15.423091,2.848433,6.183781,6.878625,-0.037441,1
...,...,...,...,...,...,...,...,...,...,...
11515,48.524351,0.869430,1.486233,0.288369,40.450824,8.815813,23.986074,19.323733,-0.067352,3
11516,54.586558,0.847464,1.513751,0.428790,55.854137,12.669003,31.871685,22.948608,-0.075988,3
11517,53.141926,0.865656,1.494438,0.336398,58.245148,11.076394,26.317758,22.568554,-0.036833,3


In [ ]:
# dfn['NLabel'] = pd.cut(x=dfn['Labels'], bins=[0, 3, 5, 8, 10],
#                      labels=['Normal Anxiety', 'Light Anxiety', 'Moderate Anxiety',
#                              'Severe Anxiety'])
# # print(d1)

# print("Categories: ")
# print(dfn['NLabel'].value_counts())
# dfn = dfn.drop('Labels',axis= 'columns')
# dfn

In [ ]:
from sklearn.svm import SVC


from sklearn.model_selection import train_test_split

X = d1.drop('NLabel', axis=1) # Features
y = d1['NLabel'] # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

svm_model = SVC(kernel='linear')

svm_model.fit(X_train, y_train)

y_pred = svm_model.predict(X_test)

from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [132]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X = d1.drop('NLabel', axis=1) # Features
y = d1['NLabel'] # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
# Here, n_estimators is the number of trees in the forest, and random_state is used to ensure reproducibility.


rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)

from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy*100)

Accuracy: 48.22048611111111


In [133]:
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split

X = d1.drop('NLabel', axis=1) # Features
y = d1['NLabel'] # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

knn_model = KNeighborsClassifier(n_neighbors=5)

knn_model.fit(X_train, y_train)
y_pred = knn_model.predict(X_test)

from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)







Accuracy: 0.3648726851851852


In [134]:
pip install scikit-optimize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [135]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from skopt import BayesSearchCV
from skopt.space import Real, Integer




X = d1.drop('NLabel', axis=1) # Features
y = d1['NLabel'] # Target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Define the search space for hyperparameters
search_space = {
    'n_estimators': Integer(10, 200),
    'max_depth': Integer(2, 20),
    'min_samples_split': Real(0.01, 1.0),
    'min_samples_leaf': Integer(1, 10),
    'max_features': ['sqrt', 'log2', None]
}

# Create an instance of the Random Forest model
rf_model = RandomForestClassifier()

# Create an instance of the BayesSearchCV class
opt = BayesSearchCV(
    rf_model,
    search_space,
    cv=5,
    n_iter=50,
    scoring='accuracy',
    n_jobs=-1,
    random_state=42
)

# Fit the BayesSearchCV class on the training data
opt.fit(X_train, y_train)

# Print the best hyperparameters and accuracy score
print("Best hyperparameters:", opt.best_params_)
print("Accuracy score:", opt.best_score_)


Best hyperparameters: OrderedDict([('max_depth', 20), ('max_features', None), ('min_samples_leaf', 2), ('min_samples_split', 0.13727017252832655), ('n_estimators', 200)])
Accuracy score: 0.4243921568627451
